In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("dataset.csv")
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [7]:
from sklearn.impute import SimpleImputer
import numpy as np

# Imputing missing values for numerical columns with median
num_imputer = SimpleImputer(strategy='median')
df[['LoanAmount', 'Loan_Amount_Term']] = num_imputer.fit_transform(df[['LoanAmount', 'Loan_Amount_Term']])

# Imputing missing values for categorical columns with mode
# First, calculate the mode (most frequent value) for each column
cat_columns = ['Gender', 'Married', 'Dependents', 'Self_Employed', 'Credit_History']
for col in cat_columns:
    df[col].fillna(df[col].mode()[0], inplace=True)

# Check if there are any missing values left
df.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [8]:
df["Loan_Status"] = (df["Loan_Status"] == "Y").astype(int)
df = df.drop(columns=["Loan_ID"], axis=1)
# In dependents column, there is a value 3+ which is not a number. So, we need to replace it with a number.
df["Dependents"] = df["Dependents"].replace("3+", 3)
df["Dependents"] = df["Dependents"].astype(float)

In [9]:
df = df.fillna(df.mean())

C:\Users\Mitkumar Paul\AppData\Local\Temp\ipykernel_24668\114435927.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df = df.fillna(df.mean())


In [10]:
# Fill missing values in binary columns with the most common value
cols = ["Gender", "Married", "Self_Employed"]
for col in cols:
    df[col] = df[col].fillna(df[col].mode()[0])

In [11]:
# Encode categorical columns
from sklearn.preprocessing import LabelEncoder
cols = ["Gender","Married","Dependents","Education","Self_Employed", "Property_Area"]
le = LabelEncoder()
for col in cols:
    df[col] = le.fit_transform(df[col])

df

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1,0,0,0,0,5849,0.0,128.0,360.0,1.0,2,1
1,1,1,1,0,0,4583,1508.0,128.0,360.0,1.0,0,0
2,1,1,0,0,1,3000,0.0,66.0,360.0,1.0,2,1
3,1,1,0,1,0,2583,2358.0,120.0,360.0,1.0,2,1
4,1,0,0,0,0,6000,0.0,141.0,360.0,1.0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
609,0,0,0,0,0,2900,0.0,71.0,360.0,1.0,0,1
610,1,1,3,0,0,4106,0.0,40.0,180.0,1.0,0,1
611,1,1,1,0,0,8072,240.0,253.0,360.0,1.0,2,1
612,1,1,2,0,0,7583,0.0,187.0,360.0,1.0,2,1


In [24]:
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

X = df.drop('Loan_Status', axis='columns', inplace=False)
y = df['Loan_Status']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

model1 = LogisticRegression(random_state=1)
model2 = RandomForestClassifier(n_estimators=200, random_state=1, max_leaf_nodes=10) 
model3 = KNeighborsClassifier(n_neighbors=5)
model4 = SVC(kernel='rbf', random_state=1)

model = VotingClassifier(estimators=[('lr', model1), ('rf', model2), ('knn', model3), ('svm', model4)], voting='hard')
model.fit(x_train,y_train)
model.score(x_test,y_test)

c:\Users\Mitkumar Paul\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7967479674796748

In [32]:
from sklearn.metrics import accuracy_score

rf_classifier = RandomForestClassifier(n_estimators=200, random_state=1, max_leaf_nodes=20)
rf_classifier.fit(x_train, y_train)
y_pred = rf_classifier.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.7967479674796748